<a href="https://colab.research.google.com/github/roseberr/data-analysis/blob/master/keggle/heart_disease/Heart_Disease_UCI_Kaggle_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats as st
import matplotlib.pyplot as plt
import sklearn as sklearn
import seaborn as sn
from sklearn.metrics import confusion_matrix
import torch
import torchvision
import torchtext


In [0]:
!git clone https://github.com/roseberr/data-analysis.git

In [0]:
heart_df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/데이터분석 /heart disease_keggle/heart.xls')

heart_df

기본적인 값 확인과 NA없는 data 확인 

In [0]:
print(heart_df.head)
print(heart_df.shape)

print(heart_df.info())
print(heart_df.isnull().sum())


In [0]:
print(heart_df.isnull().sum())#이상치 개수 확인
print(heart_df.dropna(axis=0,inplace=True))#na인 행 제거

histogram 그리기 

In [0]:
def draw_histograms(dataframe, features, rows, cols):
    fig = plt.figure(figsize=(10, 10))
    for i, feature in enumerate(features):
        ax = fig.add_subplot(rows, cols, i + 1)
        dataframe[feature].hist(bins=20, ax=ax, facecolor='midnightblue')
        ax.set_title(feature + " Distribution", color='DarkRed')
    fig.tight_layout()
    plt.show()
draw_histograms(heart_df,heart_df.columns,6,3) #histogram 출력

한국말 패치


In [0]:
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()


sex count graph

In [0]:
print(heart_df.describe())
sn.barplot(x="sex", y="target", data=heart_df)
sn.countplot(x="sex",data=heart_df)

plt.title("요일 별, 전체 팁")
plt.show()

heart_df['sex'] = heart_df.sex.map({0: 'female', 1: 'male'})##이름 rename

sex와 target의 상관그래프

In [0]:
sn.set_context("paper", font_scale = 1, rc = {"font.size": 20,"axes.titlesize": 15,"axes.labelsize": 30})
sn.catplot(kind = 'count', data = heart_df, x = 'sex', hue = 'target', order = heart_df['sex'].sort_values().unique())
plt.title('Variation of Sex for each target class')
plt.show()

age와 target의 관계

In [0]:
sn.catplot(kind = 'count', data = heart_df,  x = 'age', hue = 'target')
plt.title('Distribution of age vs sex with the target class')
plt.show()

성별과 나이의 밀도

In [0]:
sn.set_context("paper", font_scale = 1, rc = {"font.size": 20,"axes.titlesize": 15,"axes.labelsize": 30})
sn.catplot(kind ='violin', data = heart_df, x = 'age', y='sex', hue = 'target')
plt.title('Variation of Age for each target class')
plt.show()

그림그리기위해서 임의로 성별 바꾼것을 원래 데이터로 변환 

In [0]:
heart_df['sex'] = heart_df.sex.map({'female': 0, 'male': 1})

p-value feature 지우기


In [0]:
from statsmodels.tools import add_constant as add_constant

heart_df_constant = add_constant(heart_df)

st.chisqprob = lambda chisq, df: st.chi2.sf(chisq, df)#카이제곱 분포사용 감마 분포의 한종류

cols=heart_df_constant.columns[:-1]

model=sm.Logit(heart_df.target,heart_df_constant[cols])

result=model.fit()
print(result.summary())

위의 결과는 P 값이 선호 알파(5%)보다 높아서 심장 질환 확률과 통계적으로 유의미한 관계가 낮은 특성을 보여준다.

여기서 후방 낙차 접근방식은 모든 속성의 P 값이 0.05 미만일 때까지 반복적으로 회귀 분석을 실행하여
한 번에 P값이 가장 높은 속성을 제거하는 데 사용된다.


In [0]:
def back_feature_elem (data_frame,dep_var,col_list):
    """ Takes in the dataframe, the dependent variable and a list of column names, runs the regression repeatedly eleminating feature with the highest
    P-value above alpha one at a time and returns the regression summary with all p-values below alpha"""

    while len(col_list)>0 :
        model=sm.Logit(dep_var,data_frame[col_list])
        result=model.fit(disp=0)
        largest_pvalue=round(result.pvalues,3).nlargest(1)
        if largest_pvalue[0]<(0.05):
            return result
            break
        else:
            col_list=col_list.drop(largest_pvalue.index)

result=back_feature_elem(heart_df_constant,heart_df.target,cols)

print(result.summary())

sex의 odds ratio에서 남성이 여성보다 심장병걸릴확률이 1-0.24만큼 낮다
odd ratio는 비율이다

In [0]:
params = np.exp(result.params)
conf = np.exp(result.conf_int())
conf['OR'] = params
pvalue=round(result.pvalues,3)
conf['pvalue']=pvalue
conf.columns = ['CI 95%(2.5%)', 'CI 95%(97.5%)', 'Odds Ratio','pvalue']
print ((conf))



data processing

수치형 스케일링 

MinMaxScaler: 가장작은값을 0 가장 큰 값을 1로 바꿔줌 이상치에 취약
standardScaler: 가우시안 분포로 만들어줌


범주형 스케일링

LaenlEncode: 값들을 0,1,2 이런 숫자로 바꿔줌

inverse_transform :역변환

OneHotEncode:희소행렬형태로 반환해줌

Preprocessing:데이터 값이 너무 작으면 svm처럼 다른 컬럼 추가하는 전처리방식

결측치의 수정,스케일의 변환 특성 추가 기능제공
sklearn.impute,SimpleImputer



In [0]:
x = heart_df.iloc[:, :-1].values
y = heart_df.iloc[:, -1].values

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0,stratify=y)

from sklearn.preprocessing import StandardScaler as ss

sc = ss()
x_train = sc.fit_transform(x_train)#스케일링 standardscaler 적용 평균이 0과 표준편차가 1이되도록 변환
x_test = sc.transform(x_test)

logistic resgression
solver: : L1제약조건,L2제약조건 2가지 모두 지원하여 이것은 작은 데이터에 적합한 알고리즘  saga 이것은  대용량 데이터에 적합한 알고리즘 

penalty: L1,L2제약조건을 설정하는 하이퍼 파라미터이고 default는 L2

class weight: 데이터에 직접 가중치를 설정하여 학습의 강도를 다르게 할수있다.

C:alpha와 다르다 높은C를 설정할수록 낮은 강도의 제약조건이 설정되고  낮은C를 설정할수록 높은 강도의 제약조건이 설정된다



L2 제약조건: 모든 특성에 대한 가중치의 값을 0주변으로 위치하도록 제어하는 제약조건

L1 제약조건: 별로 중요하지 않는 칼럼의 계수를 0으로 바꾸기

In [0]:
from sklearn.linear_model import LogisticRegression
classifier=LogisticRegression()
classifier.fit(x_train,y_train)

y_pred = classifier.predict(x_train)
print('Accuracy for train set for logistic regression={}'.format(sklearn.metrics.accuracy_score(y_train,y_pred)))


y_pred=classifier.predict(x_test)
print('Accuracy for test set for logistic regression={}'.format(sklearn.metrics.accuracy_score(y_test,y_pred)))
print()
#로지스틱 회귀 heatmap
cm=confusion_matrix(y_test,y_pred)
conf_matrix=pd.DataFrame(data=cm,columns=['Predicted:0','Predicted:1'],index=['Actual:0','Actual:1'])
plt.figure(figsize = (8,5))
sn.heatmap(conf_matrix, annot=True,fmt='d',cmap="YlGnBu")

SVM

C:제약조건 강도설정

gamma=가까이 있는 데이터의 가중치(높은 감마-> 좁은 영역에 큰 점수/낮은 감마->넓은 영역까지 접수)

max_iter:최대 반복 획수를 지정한다

class_weight:학습 분량 조절

multi_class:3개이상 분류 

penalty: 제약조건 설정


In [0]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf')
classifier.fit(x_train, y_train)

# Predicting the Test set results

y_pred = classifier.predict(x_train)
print('Accuracy for train set for svm={}'.format(sklearn.metrics.accuracy_score(y_train,y_pred)))

y_pred = classifier.predict(x_test)
print('Accuracy for test set for svm={}'.format(sklearn.metrics.accuracy_score(y_test,y_pred)))

Naive Bayes 

In [0]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(x_train, y_train)

# Predicting the Test set results

y_pred = classifier.predict(x_train)
print('Accuracy for train set for Naive Bayes  ={}'.format(sklearn.metrics.accuracy_score(y_train,y_pred)))

y_pred = classifier.predict(x_test)
print('Accuracy for test set for Naive Bayes ={}'.format(sklearn.metrics.accuracy_score(y_test,y_pred)))

Decision Tree(과적합이 심함, 데이터 범위 바깥 데이터가 들어오면 쓸모없음)

max_depth=끝까지 학습하지 못하게하며 일반화 성능 높인다

min_sample_split: 이놈이 계속 split 하면서 깊숙하게 들어가게하는데 만약 값이 2이면 한 노드에서 2개이하 데이터를 가지면 학습 멈추고 그만 들어가게함

max_feature=많은 특성들중 최대 요만큼만 사용하라 라는 파라미터


In [0]:
from sklearn.tree import DecisionTreeClassifier
import numpy as np

classifier = DecisionTreeClassifier(random_state=50)
classifier.fit(x_train, y_train)


# Predicting the Test set results
y_pred = classifier.predict(x_train)

print('Accuracy for train set for Decision Tree ={}'.format(sklearn.metrics.accuracy_score(y_train,y_pred)))

y_pred = classifier.predict(x_test)
print('Accuracy for test set for Decision Tree ={}'.format(sklearn.metrics.accuracy_score(y_test,y_pred)))


Random Forest(begging 앙상블 사용)
전처리에 영향을 1도 안받음



In [0]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10)
classifier.fit(x_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(x_train)
print('Accuracy for train set for Random Forest ={}'.format(sklearn.metrics.accuracy_score(y_train,y_pred)))

y_pred = classifier.predict(x_test)
print('Accuracy for test set for Random Forest ={}'.format(sklearn.metrics.accuracy_score(y_test,y_pred)))

앙상블기법

voteing: 하나의 데이터셋을 가지고 여러가지 머신러닝 클래스에게 물어봐서 과반수 투표

bagging:여러가지 데이터셋을 가지고 1개의 알고리즘으로 여러개의 예측기 생성

boosting:취합을 하지 않고 점진적으로 학습시키면서 연결한다.
첫번째에게 물어봐서 틀린것 보완 두번째에게서 틀린것 보완 이런식으로 점진적인 방법을 boosting이라고함

가중치를 높여가면서 하는 adaboost가 있고 틀린 오차를 학습시키는 gradient boosting이 있음



Voting 앙상블 with KFold


In [0]:
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

knn_model=KNeighborsClassifier(n_neighbors=5)
svc_model=SVC()
dt_model=DecisionTreeClassifier()
lr_model=LogisticRegression()

ensemble=VotingClassifier(estimators=[('lr',lr_model),
                                      ('knn',knn_model),
                                      ('svc',svc_model),
                                      ('dt',dt_model)])


lr_model.fit(x_train,y_train)
knn_model.fit(x_train,y_train)
svc_model.fit(x_train,y_train)
dt_model.fit(x_train,y_train)
ensemble.fit(x_train,y_train)


kfold=KFold(n_splits=5,shuffle=True)
ensemble_scores=cross_val_score(ensemble,x,y,cv=kfold,n_jobs=-1)

print('학습 평가(lr):',lr_model.score(x_test,y_test))
print('학습 평가(knn):',knn_model.score(x_test,y_test))
print('학습 평가(svc):',svc_model.score(x_test,y_test))
print('학습 평가(dt):',dt_model.score(x_test,y_test))
print('학습 평가(ensemble):',ensemble.score(x_test,y_test))
print('kfold 학습평가(ensemble)',ensemble_scores.mean())

Boosting 앙상블

weaker가 stronger가 되어갈때 데이터의 가중치를 조절해 가면서 stringer가 되어가는 Adaboost

weaker가 stronger가 되어갈 때 잔여 오차를 재차 학습하면 stronger가 되어가는 GradientBoost가 있다.

adaboost는 내부 예측기를 지정해서 돌리지만 gradientboost는 내부예측기는 decision tree사용


In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

#adaboost
base_model=DecisionTreeClassifier(
    max_depth=3,
    max_features=0.2,
    class_weight='balanced',
    random_state=1
)

ada_model=AdaBoostClassifier(
    base_estimator=base_model,
    n_estimators=1000,
    learning_rate=1.,
    random_state=1
).fit(x_train,y_train)

print('정확도(ada_model):',ada_model.score(x_test,y_test))

#gradient boosting
from sklearn.ensemble import GradientBoostingClassifier

gb_model=GradientBoostingClassifier(
    learning_rate=0.1,
    n_estimators=10000,
    max_depth=3,#보통 5이상 두지 않는다
    max_features=0.6,
    subsample=0.7, #과적합 방지 하이퍼 파라미터
    random_state=1).fit(x_train,y_train)

print('정확도(gb_model):', gb_model.score(x_test,y_test))

print("feature importance::",gb_model.feature_importances_)

stacking 앙상블

In [0]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
def get_stacking_data(model, x_train,y_train,x_test,n_folds=5):
    kfold=KFold(n_splits=n_folds,random_state=0,shuffle=True)
    train_fold_predict=np.zeros((x_train.shape[0],1))
    test_predict=np.zeros((x_test.shape[0],n_folds))
    print('model:',model.__class__.__name__)

    for cnt, (train_index,valid_index) in enumerate(kfold.split(x_train)):
        x_train_ = x_train[train_index]
        y_train_=y_train[train_index]
        x_validation=x_train[valid_index]

        model.fit(x_train_,y_train_)
        train_fold_predict[valid_index,:]=model.predict(x_validation).reshape(-1,1)
        test_predict[:,cnt]=model.predict(x_test)

    test_predict_mean=np.mean(test_predict,axis=1).reshape(-1,1)

    return train_fold_predict,test_predict_mean

svm = SVC(kernel = 'rbf')
rf = RandomForestClassifier(n_estimators = 10)
lr=LogisticRegression()

svm_train,svm_test=get_stacking_data(svm,x_train,y_train,x_test)
rf_train,rf_test=get_stacking_data(rf,x_train,y_train,x_test)
lr_train,lr_test=get_stacking_data(lr,x_train,y_train,x_test)

new_x_train=np.concatenate((svm_train,rf_train,lr_train),axis=1)
new_x_test=np.concatenate((svm_test,rf_test,lr_test),axis=1)

knn=KNeighborsClassifier(n_neighbors=2,n_jobs=-1)     
#knn은 scaling을 사용하고 해야한다 knn이 거리기반이어서 범위가 같아야한다 

knn.fit(new_x_train,y_train)
stack_pred=knn.predict(new_x_test)
print("정확도: ",(sklearn.metrics.accuracy_score(stack_pred,y_test)))